<a href="https://colab.research.google.com/github/DamarNovtahaning/Deep-Learning/blob/main/InceptionV3_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import tensorflow as tf
import tensorflow.keras as K
import matplotlib.pyplot as plt
import numpy as np

In [27]:
(X_train, Y_train), (X_test, Y_test) = K.datasets.cifar10.load_data()

In [28]:
Y_train.shape

(50000, 1)

In [29]:
Y_train[0]

array([6], dtype=uint8)

In [30]:
def preprocessing_data(X, Y):
  x = tf.keras.applications.inception_v3.preprocess_input(X)
  y = K.utils.to_categorical(Y, 10)
  return x, y

In [31]:
X_train, Y_train = preprocessing_data(X_train, Y_train)
X_test, Y_test = preprocessing_data(X_test, Y_test)

In [32]:
Y_train[0]

array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.], dtype=float32)

In [33]:
base_model = K.applications.InceptionV3(
    include_top=False,
    weights='imagenet',
    input_shape=(299, 299, 3),
    classifier_activation='softmax'
)

In [34]:
base_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_94 (Conv2D)             (None, 149, 149, 32  864         ['input_5[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_94 (BatchN  (None, 149, 149, 32  96         ['conv2d_94[0][0]']              
 ormalization)                  )                                                      

In [35]:
inputs = K.Input(shape=(32,32,3))
input = K.layers.Lambda(lambda image : tf.image.resize(image, (299, 299)))(inputs)

x = base_model(input, training=False)

x = K.layers.GlobalAveragePooling2D()(x)

x = K.layers.Dense(300, activation='relu')(x)
x = K.layers.Dropout(0.3)(x)
outputs = K.layers.Dense(10, activation='softmax')(x)

model = K.Model(inputs, outputs)

In [36]:
lr_schedule = K.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=1000,
    decay_rate=0.5)

In [37]:
base_model.trainable = False
optimizer = K.optimizers.SGD(learning_rate=lr_schedule)

model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["acc"])

In [ ]:
history = model.fit(X_train, Y_train, 
                    validation_data=(X_test, Y_test), 
                    batch_size=128, epochs=10, verbose=1)

Epoch 1/10
391/391 [==============================] - 452s 1s/step - loss: 2.0112 - acc: 0.3181 - val_loss: 1.6271 - val_acc: 0.6312
Epoch 2/10
 10/391 [..............................] - ETA: 5:52 - loss: 1.6871 - acc: 0.5000

In [ ]:
results = model.evaluate(X_test, Y_test)
print("test loss, test acc : ", results)

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(base_model, to_file='model.png')

In [ ]:
for i, layer in enumerate(base_model.layers):
  print(i, layer.name)

In [ ]:
for layer in base_model.layers[:150]:
  layer.trainable=False

for layer in base_model.layers[150:]:
  layer.trainable=True

In [ ]:
optimizer=K.optimizers.SGD(1e-3)
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["acc"])

In [ ]:
history = model.fit(X_train, Y_train, 
                    validation_data=(X_test, Y_test), 
                    batch_size=128, epochs=10, verbose=1)